In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
MODEL = "mistral"
MODEL = "mixtral:8x7b"

In [2]:
import os


from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

# Load environment variables from .env file
load_dotenv()

# Get your OpenAI API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the model you want to use
MODEL = "mistral"   # or "gpt-3.5-turbo"

# Switch based on model name
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

# Invoke a prompt
model.invoke("Tell me a joke")



" Here's one for you: Why don't scientists trust atoms?\n\nBecause they make up everything!\n\nI hope that brings a smile to your face. If you need anything else or have any other requests, feel free to ask!"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

" Sure! Here's one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything... except the periodic table!\n\nHope that made you smile! If you have any other requests or questions, feel free to ask."

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [6]:
chain = prompt | model | parser

chain.invoke({"context": "people call me Rudraks", "question": "What's your name'?"})

' Rudraks'

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("doc.pdf")
pages = loader.load_and_split()
pages

invalid pdf header: b'\xd9/\x00\x00%'
incorrect startxref pointer(1)
parsing for Object Streams


[Document(metadata={'source': 'doc.pdf', 'page': 0}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 3 \nPETITIONER:\nK.M. SHARMA\n        Vs.\nRESPONDENT:\nDEVI LAL & ORS.\nDATE OF JUDGMENT09/01/1990\nBENCH:\nMISRA RANGNATH\nBENCH:\nMISRA RANGNATH\nPUNCHHI, M.M.\nCITATION:\n 1990 AIR  528            1990 SCR  (1)   1\n 1990 SCC  (1) 438        JT 1990 (1)     5\n 1990 SCALE  (1)1\nACT:\n    Constitution of India, 1950: Article 75(4) and  Schedule\n111  Forms I and II--Oath/Affirmation by  Minister--Descrip-\ntion  of  person as Deputy  Prime  Minister--Whether  valid,\nlegal and constitutional.\nHEADNOTE:\n    The petitioner challenged the appointment of  Respondent\nNo.  1 as Deputy Prime Minister of India on the ground  that\nthe  oath  administered to him as such was not the  oath  in\naccordance with the prescription of the Constitution.\n    The  petition was contested by Respondent No. 1 and  the\nUnion  of  India. It was contended for the latter  that  de-\ns

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [9]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(metadata={'source': 'doc.pdf', 'page': 2}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 3 of 3 \ncerned, the petitioner himself does not press it.\n    We accordingly reject the writ petition for the  reasons\nindicated above. No costs.\nP.S.   S                                            Petition\ndismissed.\n4'),
 Document(metadata={'source': 'doc.pdf', 'page': 0}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 3 \nPETITIONER:\nK.M. SHARMA\n        Vs.\nRESPONDENT:\nDEVI LAL & ORS.\nDATE OF JUDGMENT09/01/1990\nBENCH:\nMISRA RANGNATH\nBENCH:\nMISRA RANGNATH\nPUNCHHI, M.M.\nCITATION:\n 1990 AIR  528            1990 SCR  (1)   1\n 1990 SCC  (1) 438        JT 1990 (1)     5\n 1990 SCALE  (1)1\nACT:\n    Constitution of India, 1950: Article 75(4) and  Schedule\n111  Forms I and II--Oath/Affirmation by  Minister--Descrip-\ntion  of  person as Deputy  Prime  Minister--Whether  valid,\nlegal and constitutional.\nHEADNOTE:\n    The petitioner cha

In [10]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
   "What are the common reasons for rejecting a legal appeal?",
    "What is the time limit to file an appeal in court?",
    "Can new evidence be introduced during an appeal?",
    "What is the difference between an appeal and a review?",
    "Who can file a legal appeal — the accused or the complainant?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What are the common reasons for rejecting a legal appeal?
Answer:  The context provided does not contain specific information about the common reasons for rejecting a legal appeal. However, in this particular case, the writ petition was rejected based on the following reasons:

1. The petitioner himself did not press the main point of contention.
2. The oath administered to the respondent as Deputy Prime Minister was not considered problematic since it did not confer any power beyond that of a regular minister. The description of him as Deputy Prime Minister was deemed as descriptive and not constitutional in nature.

Question: What is the time limit to file an appeal in court?
Answer:  I don't know. The provided context does not contain information about the time limit for filing an appeal in court.

Question: Can new evidence be introduced during an appeal?
Answer:  The provided context does not discuss the introduction of new evidence during an appeal. Therefore, I don't k